# Homework 3

```yaml
Course:   DS 5001 
Module:   03 Language Models
Topic:    Homework 3
Author:   Ryan Lipps
Date:     2/1/2024
```

In [1]:
import numpy as np
import pandas as pd
import textimporter

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
text_file = f"{data_home}/gutenberg/pg42324.txt"

In [4]:
ohco_pats = [('chap', r'^(?:LETTER|CHAPTER)\b', 'm')]
clip_pats = [r'START', r'END']
timporter = textimporter.TextImporter(src_file=text_file, ohco_pats=ohco_pats, clip_pats=clip_pats)
timporter.import_source().parse_tokens(special_tokens=['_'])
print(timporter.TOKENS.head())
print(timporter.gather_tokens(1))

Importing  /Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(?:LETTER|CHAPTER)\b
Parsing OHCO level 1 para_num by delimiter \n\n
Parsing OHCO level 2 sent_num by delimiter [.?!;:]+
Parsing OHCO level 3 token_num by delimiter [\s',-,_,--]+
                                    token_str term_str
chap_id para_num sent_num token_num                   
1       0        0        1                To       to
                          2               Mrs      mrs
                 1        1           Saville  saville
                          2           England  england
        1        0        0                St       st
                                                       para_num_str
chap_id para_num                                                   
1       0                                    to mrs saville england
        1                                st petersburgh dec 11th 17
        2         you will rej

/Users/ryanlipps/miniconda3/envs/ds-6001/lib/python3.10/site-packages/pandas/core/strings/object_array.py:343: FutureWarning: Possible set difference at position 9
  new_pat = re.compile(pat)


In [8]:
TOKENS = timporter.TOKENS
TOKENS.head(30)

token_str      term_str
chap_id para_num sent_num token_num                            
1       0        0        1                    To            to
                          2                   Mrs           mrs
                 1        1               Saville       saville
                          2               England       england
        1        0        0                    St            st
                 1        1           Petersburgh   petersburgh
                          2                   Dec           dec
                 2        1                  11th          11th
                          2                    17            17
        2        0        0                   You           you
                          1                  will          will
                          2               rejoice       rejoice
                          3                    to            to
                          4                  hear          hear
                          5                  that          that
                          6                    no            no
                          7              disaster      disaster
                          8                   has           has
                          9           accompanied   accompanied
                          10                  the           the
                          11         commencement  commencement
                          12                   of            of
                          13                   an            an
                          14           enterprise    enterprise
                          15                which         which
                          16                  you           you
                          17                 have          have
                          18             regarded      regarded
                          19                 with          with
                          20                 such          such